In [1]:
from __future__ import division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import fetch_mldata
import sklearn.datasets as sk_data
import math
import time
#d afdsf
mnist = fetch_mldata('MNIST original', data_home=sk_data.get_data_home())

IMG_PIX = mnist.data.shape[1]
IMG_SIZE = int(math.sqrt(IMG_PIX))
TRAINING_SIZE = 60000
TEST_SIZE = 10000
TOTAL_SIZE = mnist.data.shape[0]
EPOCH = 50
BATCH_SIZE = 100
WINDOW_SIZE = 10000/BATCH_SIZE

print "IMG_PIX: %s\nIMG_SIZE: %s\nTOTAL_SIZE: %s\n"%(IMG_PIX, IMG_SIZE, TOTAL_SIZE)

def vectorize_y(Y):
    v_y = np.zeros((Y.shape[0], max(set(Y.flatten()))+1))
    print v_y.shape
    for i in range(len(Y)):
        v_y[i][Y[i]] = 1.0
    return v_y

def shuffle(X, Y):
    Y = np.array([Y.flatten()]).T
    A = np.concatenate((X, Y),axis=1)
    np.random.shuffle(A)
    X_prime, Y_prime = np.hsplit(A, [-1])
#     if (set(Y_prime.flatten())!=set([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0])):
#         return shuffle(X,Y)
    Y_prime = Y_prime.reshape(Y_prime.shape[0],)
    return X_prime, Y_prime

def prepare_data(_mnist):
    _mnist.data = _mnist.data.astype(np.float32)
    _mnist.data = np.multiply(_mnist.data, 1.0 / 255.0)
    _mnist.target = _mnist.target.astype(np.uint8)
    train_xx, test_xx = np.vsplit(_mnist.data, [TRAINING_SIZE])
    train_yy, test_yy = np.array_split(_mnist.target, [TRAINING_SIZE])
    return train_xx, test_xx, train_yy, test_yy

print type(mnist.target[0])

IMG_PIX: 784
IMG_SIZE: 28
TOTAL_SIZE: 70000

<type 'numpy.float64'>


In [2]:
import fftf
import ffbn

HIDDEN_1 = 64
HIDDEN_2 = 64
ALPHA = 0.5
USING_BATCH_NORM = False
if (USING_BATCH_NORM):
    import ffbn as ff
else: 
    import fftf as ff

train_x, test_x, train_y, test_y = prepare_data(mnist)
print "training =>", train_x.shape, train_y.shape
print "testing =>", test_x.shape, test_y.shape
print ff

with tf.Graph().as_default(), tf.device('/gpu:0'):      
    img_input = tf.placeholder(tf.float32, shape=[BATCH_SIZE, IMG_PIX])
    label_input = tf.placeholder(tf.int32, shape=[BATCH_SIZE])
    model = None
    if (USING_BATCH_NORM):
        model = ff.inference(img_input, HIDDEN_1, HIDDEN_2, batch_norm_ver=None)
    else:
        model = ff.inference(img_input, HIDDEN_1, HIDDEN_2)
    loss_f = ff.loss(model, label_input)
    training_model = ff.training(loss_f, ALPHA)
    eval_f = ff.evaluation(model, label_input)
    init = tf.initialize_all_variables()
    
    loss_arr = []
    eval_arr = []
    moving_done = False
    
    done = False
    
    with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
        sess.run(init)
        t0 = time.time()
        moving_idx = 0
        moving_avg = np.zeros(WINDOW_SIZE)
        step = 0
        for i in xrange(EPOCH):
            print "EPOCH:", i+1
            acc_sum = 0
            train_x, train_y = shuffle(train_x, train_y)
            for j in xrange(0, TRAINING_SIZE, BATCH_SIZE):
                train_x_batch = train_x[j:j+BATCH_SIZE]
                train_y_batch = train_y[j:j+BATCH_SIZE]
                _, loss, correct = sess.run([training_model, loss_f, eval_f], feed_dict={
                    img_input: train_x_batch,
                    label_input: train_y_batch
                })
                
                moving_avg[moving_idx] = correct/len(train_x_batch)
                moving_idx += 1
                if (moving_idx >= WINDOW_SIZE):
                    moving_idx = 0
                    moving_done = True
                
                wind_avg= moving_avg.sum()/WINDOW_SIZE
                
                if(j%1000)==0 or j+BATCH_SIZE == TRAINING_SIZE:
                    print "(STEP %s) loss= %s acc= %s accum_acc= %s t= %s sec"%(step, loss, correct/BATCH_SIZE, wind_avg, time.time()-t0)
                
                if (moving_done and wind_avg >= 0.97):
                    print "-----------------"
                    print "(STEP %s) loss= %s acc= %s accum_acc= %s t= %s sec"%(step, loss, correct/BATCH_SIZE, wind_avg, time.time()-t0)
                    done = True
                    break;
                step+=1
                    
            if (done):
                break;
        num_correct = 0
        for k in xrange(0, TEST_SIZE, BATCH_SIZE):
            eval_x, eval_y = test_x[k:k+BATCH_SIZE], test_y[k:k+BATCH_SIZE]
            if (len(eval_x)!=BATCH_SIZE): 
                break
            correct = sess.run(eval_f, feed_dict={
                img_input: eval_x,
                label_input: eval_y
            })
            num_correct += correct
#                 print num_correct, correct
        print "\tTEST ACCURACY= %s"%(num_correct/TEST_SIZE)
                    
                



training => (60000, 784) (60000,)
testing => (10000, 784) (10000,)
<module 'fftf' from 'fftf.pyc'>
Instructions for updating:
Use `tf.global_variables_initializer` instead.


InvalidArgumentError: Cannot assign a device to node 'global_step': Could not satisfy explicit device specification '/device:GPU:0' because no supported kernel for GPU devices is available.
Colocation Debug Info:
Colocation group had the following types and devices: 
AssignAdd: CPU 
Const: GPU CPU 
Assign: CPU 
Identity: CPU 
VariableV2: CPU 
	 [[Node: global_step = VariableV2[container="", dtype=DT_INT32, shape=[], shared_name="", _device="/device:GPU:0"]()]]

Caused by op u'global_step', defined at:
  File "/home/bladestery/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/bladestery/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/bladestery/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/bladestery/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in l

In [ ]:
# from tensorflow.examples.tutorials.mnist import input_data


# ds = input_data.read_data_sets("data", validation_size=0)

# print ds.train.epochs_completed
# print ds.train.images.shape

# print train_y[0]==ds.train.labels[1]
# print set(ds.train.images[1]) == set(train_x[0])

# plt.subplot(3,3,1)
# plt.imshow(train_x[0].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,2)
# plt.imshow(train_x[1].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,3)
# plt.imshow(train_x[2].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,4)
# plt.imshow(train_x[3].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,5)
# plt.imshow(train_x[4].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,6)
# plt.imshow(train_x[5].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,7)
# plt.imshow(train_x[6].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,8)
# plt.imshow(train_x[7].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,9)
# plt.imshow(train_x[8].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')

# # plt.subplot(1,2,2)
# # plt.imshow(ds.train.images[3].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# # print train_y[2]
# # print ds.train.labels[3]

# print type(ds.train.labels[0])
# print train_y[:10]





